In [1]:
import os 
import pandas as pd

In [2]:
path = os.getcwd()
path = path + '\dataset'
df = pd.read_csv(path + '\Fake news classifier train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df.shape

(20800, 5)

In [5]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [6]:
df = df.dropna()

In [7]:
df.shape

(18285, 5)

In [8]:
## get the independent feautures
X = df.drop('label', axis=1)
y = df['label']

In [9]:
## Checkking whether dataset is balanced or not
y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [10]:
import tensorflow as tf
print(tf.__version__)

2.11.0


In [11]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense## for creating nuerons
from tensorflow.keras.layers import Bidirectional

In [12]:
voc_size = 5000

In [13]:
messages=X.copy()
messages.reset_index(inplace=True) ##required because we removed empty rows

In [14]:
messages.head()

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [15]:
import nltk
import re

In [16]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lmbmo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:
from nltk.corpus import stopwords

In [19]:
## Data preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(len(messages)):
    review = re.sub('[^a-zA-Z]',' ', messages['title'][i])
    review = review.lower().split()
    
    review = [ps.stem(word) for word in review if word not in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [20]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [22]:
# one hot encodding
onehot_repr = [one_hot(words, voc_size) for words in corpus]
onehot_repr

[[3856, 2558, 4950, 1998, 3051, 1894, 4131, 3893, 2944, 2524],
 [3483, 2411, 4493, 1253, 3419, 4972, 2443],
 [1576, 1779, 2720, 4575],
 [4133, 1469, 2821, 3072, 302, 1513],
 [1927, 3419, 2411, 1150, 3109, 4403, 3419, 1187, 1630, 3332],
 [4333,
  3458,
  1915,
  1475,
  1402,
  3322,
  4045,
  3389,
  3811,
  1763,
  149,
  1710,
  1492,
  768,
  2443],
 [562, 1010, 4764, 842, 272, 3998, 596, 1011, 280, 2425, 2551],
 [2789, 2054, 406, 4628, 3637, 4118, 3322, 1065, 280, 2425, 2551],
 [1774, 3741, 3483, 4356, 2756, 3204, 2024, 3393, 3322, 414],
 [3145, 2177, 1498, 1899, 347, 4103, 868, 2618],
 [1185, 3824, 2454, 1350, 241, 918, 4927, 1173, 3946, 2194, 3753],
 [3072, 1886, 3051, 3204, 3322, 3637],
 [1086, 1492, 3629, 1610, 3067, 1232, 89, 4048, 1162],
 [4045, 1489, 1758, 3808, 1237, 2687, 3788, 280, 2425, 2551],
 [890, 1224, 2492, 392, 3114, 280, 2425, 2551],
 [1267, 1912, 2424, 4203, 1081, 1921, 4530, 4626, 359, 4234],
 [2977, 4181, 2411],
 [4482, 1346, 3675, 1840, 3322, 2949, 3754, 2443]

In [24]:
print(corpus[0])
print(onehot_repr[0])

hous dem aid even see comey letter jason chaffetz tweet
[3856, 2558, 4950, 1998, 3051, 1894, 4131, 3893, 2944, 2524]


In [ ]:
## Embedding representation
sent_length = 20
